In [1]:
import json
import torch
import pandas as pd

torch.__version__

'2.1.1+cu121'

# モデル

In [2]:
model_path = "cyberagent/calm2-7b-chat"

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from transformers import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    use_flash_attention_2=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# データ

In [4]:
url_tmpl = "https://raw.githubusercontent.com/SkelterLabsInc/JaQuAD/main/data/train/jaquad_train_{i:04d}.json"
jaquad = pd.concat([pd.read_json(url_tmpl.format(i=i)) for i in range(30)], ignore_index=True)
jaquad.head(1)

,version,data
0,JaQuAD-version-0.1.0,"{'title': '手塚治虫', 'paragraphs': [{'context': '..."


In [5]:
jaquad.shape

(691, 2)

In [6]:
all_qa = []
for idx, row in jaquad.iterrows():
    title = row["data"]["title"]
    for para in row["data"]["paragraphs"]:
        context = para["context"]
        for qa in para["qas"]:
            q = qa["question"]
            a = qa["answers"][0]["text"]
            assert len(qa["answers"]) == 1
            all_qa.append([q, a])
len(all_qa)

31748

# Few-Shot テンプレート

In [7]:
TMPL = """

USER: 質問への回答をチャット形式に変換せよ
質問: 日本では改名には何の許可が必要ですか？
回答: 家庭裁判所
ASSISTANT: 日本では、改名するには家庭裁判所の許可が必要です。<|endoftext|>

USER: 質問への回答をチャット形式に変換せよ
質問: モーリシャスの信仰で最も多いのは
回答: ヒンドゥー教
ASSISTANT: モーリシャスで最も信仰されている宗教はヒンドゥー教です。<|endoftext|>

USER: 質問への回答をチャット形式に変換せよ
質問: 核反応を利用した電池を何と呼ぶ
回答: 「原子力電池」
ASSISTANT: 核反応を利用した電池は「原子力電池」と呼ばれます。<|endoftext|>

USER: 質問への回答をチャット形式に変換せよ
質問: カール・マルクスの妻イェニーが、亡くなったのはいつか。
回答: 1881年12月2日
ASSISTANT: イェニーは1881年12月2日に亡くなりました。<|endoftext|>

USER: 質問への回答をチャット形式に変換せよ
質問: 北京市、上海市、深圳市と共に何に分類されている？
回答: 一線都市
ASSISTANT: 北京市は、一線都市に分類されています。<|endoftext|>

USER: 質問への回答をチャット形式に変換せよ
質問: 武田薬品工業の取締役は何名か
回答: 13名
ASSISTANT: 取締役は13名で構成されています。<|endoftext|>

USER: 質問への回答をチャット形式に変換せよ
質問: 辺野古移設のための埋め立て工事に賛成何%？
回答: 18.99%
ASSISTANT: 18.99%の人が埋め立て工事に賛成しています。<|endoftext|>

USER: 質問への回答をチャット形式に変換せよ
質問: {q}
回答: {a}
ASSISTANT: """

# 生成

In [8]:
# 改行を含むトークンをbad_wordsに指定する
newline_ids = [[i] for i in range(65000) if "\n" in tokenizer.decode(i)]
len(newline_ids)

111

In [9]:
def generate(prompt, temperature=0.8):
    with torch.no_grad():
        input_ids = tokenizer.encode(
            prompt, return_tensors="pt", add_special_tokens=False
        ).to("cuda")
        gen_tokens = model.generate(
            max_new_tokens=128,
            input_ids=input_ids,
            do_sample=True,
            top_p=0.75,
            temperature=temperature,
            num_return_sequences=1,
            bad_words_ids=newline_ids,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    for gen_text in tokenizer.batch_decode(
        [tokens[len(input_ids[0]) :] for tokens in gen_tokens],
        skip_special_tokens=False,
    ):
        break
    return gen_text

In [ ]:
from tqdm.auto import tqdm

retrycnt = 0

for i, row in tqdm(enumerate(all_qa)):
    with open("./result.jsonl", "a", encoding="utf-8") as f:
        q = row[0]
        a = row[1]
        prompt = TMPL.format(q=row[0], a=row[1])
        while True:
            out = generate(prompt, temperature=0.8+retrycnt*0.1).replace(tokenizer.eos_token, "")

            # 以下の場合、temperatureを0.1上げてリトライする
            # 1. 改行を含む
            # 2. USERを含む（次のサンプルを生成しようとしている場合）
            # 3. 150文字以上
            # 4. 長さが元の質問以下
            if "\n" in out or "USER" in out or len(out) > 150 or len(out) <= len(a):
                retrycnt += 1
                if retrycnt > 25:
                    # どうしても出来ないときは諦める
                    break
            else:
                retrycnt = 0
                break
        f.write(json.dumps({"question": q, "answer": a, "answer_chat": out}, ensure_ascii=False))
        f.write("\n")